Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

## Deploy YOLO Real-time Object Detection using ONNX on AzureML

Prototype for Gallery scenario

### Download TinyYOLO Models

In [ ]:
import time

start = time.time()
!wget -nc -P model/ https://s3-us-west-2.amazonaws.com/coreml-models/TinyYOLO.mlmodel

end = time.time()
print("Model download latency: {} seconds".format(end-start))

### Install Dependency

*TODO*: How to do it via AZ NB Env setup?

In [ ]:
!pip install --upgrade onnxmltools
!pip install --upgrade coremltools

#### Convert TinyYOLO from CoreML model to ONNX model

*TODO*: currently have an issue that "import onnxmltools" fails due to notebook incorrectly recognizes typing package version to be 3.6.2. The current workaround is to restart the kernal. I am troubleshooting this with AZ NB team meanwhile.

In [1]:
import onnxmltools
import coremltools
import time
import os

start = time.time()

# Load a CoreML model
coreml_model = coremltools.utils.load_spec('model/TinyYOLO.mlmodel')

# Convert from CoreML into ONNX
onnx_model = onnxmltools.convert_coreml(coreml_model, 'TinyYOLOv2')

# Save ONNX model
onnxmltools.utils.save_model(onnx_model, 'model/tinyyolov2.onnx')

print(os.path.getsize('model/tinyyolov2.onnx'))

end = time.time()
print("Model conversion latency: {} seconds".format(end-start))

63480743
Model conversion latency: 80.82259154319763 seconds


### Load AML Workspace

In [2]:
from azureml.core import Workspace
ws = Workspace.from_config()
print(ws.name, ws.subscription_id, ws.resource_group, ws.location)

Found the config file in: /home/nbuser/library/aml_config/config.json
ONNX_YOLO_WS bc69d98c-7d2b-4542-88a4-f86eb4aea4a5 AndyXu_Test_RP eastus2


### Register Tiny YOLO Model to MMS


In [3]:
from azureml.core.model import Model

start = time.time()

model = Model.register(
    workspace = ws,
    model_path = "model/tinyyolov2.onnx",
    model_name = "tinyyolov2",
    tags = {"onnx": "demo"})

end = time.time()
print("Register model latency: {} seconds".format(end-start))
print("Model id={}, name={}, created time={}".format(
    model.id, 
    model.name, 
    model.created_time))

Registering model tinyyolov2
Register model latency: 82.33424162864685 seconds
Model id=tinyyolov2:1, name=tinyyolov2, created time=2018-10-17 18:37:19.473745+00:00


### Build Container Image 

In [4]:
from azureml.core.image import Image, ContainerImage

image_config = ContainerImage.image_configuration(
    execution_script = "score.py",
    runtime = "python", 
    dependencies = ["app.py", "ui.html", "static/bundle.js"],
    conda_file = "env.yml",
    tags = {"onnx": "demo"})

start = time.time()
image = Image.create(
    workspace = ws,
    name = "onnx-yolo-demo-image",
    models = [model],
    image_config = image_config
)

image.wait_for_creation(show_output = True)

end = time.time()
print("Image creation latency: {} seconds".format(end-start))
print(image)

Creating image
Running..............................
SucceededImage creation operation finished for image onnx-yolo-demo-image:1, operation "Succeeded"
Image creation latency: 216.7919270992279 seconds
ContainerImage(workspace=<azureml.core.workspace.Workspace object at 0x7ff9c854f400>, name=onnx-yolo-demo-image, id=onnx-yolo-demo-image:1, tags={'onnx': 'demo'}, properties={}, version=1)


### Deploy Image to ACI

In [5]:
from azureml.core.webservice import AciWebservice, Webservice

aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

start = time.time()
aci_service = Webservice.deploy_from_image(
    workspace = ws,
    name = "onnx-yolo-demo-service",
    image = image,
    deployment_config = aci_config)
aci_service.wait_for_deployment(show_output = True)

end = time.time()
print("Service deployment latency: {} seconds".format(end-start))

Creating service
Running..................................
SucceededACI service creation operation finished, operation "Succeeded"
Service deployment latency: 232.32787942886353 seconds


### Run Inference

*TODO*: Make it UX friendly via either run a script with sample data or let us consume the UI page


In [8]:
# Get demo URL 
demo_uri = aci_service.scoring_uri.replace("/score", "/ui")
print("Try out the ONNX TinyYOLO model at {}".format(demo_uri))

Try out the ONNX TinyYolo model at http://137.135.118.28:80/score


### Cleanup Resources

Delete the web serivce, image, and models

In [ ]:
aci_service.delete()
image.delete()
model.delete()
print("Successfully deleted models, image and service.")